# Clase 14:

## Parte 1

En esta clase aprenderemos: 
- Usar python
- Integrar Python con ArcGIS
- Usar Chat GPT para ambas cosas



In [1]:
import platform

# Print the Python version
print("Python version:", platform.python_version())


Python version: 3.9.11


In [2]:
import os

# Obtener el directorio de trabajo actual
current_directory = os.getcwd()

print("Directorio actual:", current_directory)


Directorio actual: C:\Users\af.camacho1169\Documents\ArcGIS\Projects\MyProject


In [3]:
# Defino directorio principal
import arcpy 
import os

gdb_path="C:/Users/salas.UANDES.000/Downloads/start/Tornado.gdb"
# Set the workspace to your geodatabase
arcpy.env.workspace = gdb_path

#Lista de las objetos dentro de mi path
feature_classes = arcpy.ListFeatureClasses()

# Loop through each feature class and add it to the current map
project = arcpy.mp.ArcGISProject("CURRENT")
map = project.listMaps()[0]  # Assuming you want to add to the first map in the project

for fc in feature_classes:
    layer = map.addDataFromPath(os.path.join(gdb_path, fc))
    print(f"Added {fc} to the map.")



TypeError: 'NoneType' object is not iterable